In [133]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Carregar o DataFrame a partir do arquivo CSV
dados_combinados = pd.read_csv('../data/dados_combinados.csv', sep=';')

# Convertendo 'DATA' para datetime
dados_combinados['DATA'] = pd.to_datetime(dados_combinados['DATA'])
dados_combinados['DIAS'] = (dados_combinados['DATA'] - dados_combinados['DATA'].min()).dt.days

# Criando colunas para sazonalidade específica
dias_sazonais = [20, 27, 4, 11, 18, 1, 15]
dados_combinados['SAZONALIDADE_ESPECIFICA'] = dados_combinados['DATA'].apply(lambda x: 1 if x.day in dias_sazonais else 0)

# Criando uma coluna para identificar o mês de julho

# Selecionando as características (features) e o alvo (target)
features = dados_combinados[['DIAS', 'VALOR_TOTAL_ITEM', 'SAZONALIDADE_ESPECIFICA']]
target = dados_combinados['QUANTIDADE']

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.05, random_state=35)

# Construindo o modelo de regressão linear
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = modelo.predict(X_test)

# Calculando o erro quadrático médio e o coeficiente de determinação
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Supondo que queremos prever a demanda para os próximos 7 dias
dias_futuros = pd.date_range(start=dados_combinados['DATA'].max() + pd.Timedelta(days=1), periods=7)
estoque_predito = modelo.predict(pd.DataFrame({
    'DIAS': (dias_futuros - dados_combinados['DATA'].min()).days,
    'VALOR_TOTAL_ITEM': dados_combinados['VALOR_TOTAL_ITEM'].mean(),
    'SAZONALIDADE_ESPECIFICA': dias_futuros.day.isin(dias_sazonais).astype(int)
}))

print('R²:', r2)
print('MSE:', mse)
print('Previsão de estoque para os próximos 7 dias:', estoque_predito)

R²: 0.7883694884217911
MSE: 0.58880311221315
Previsão de estoque para os próximos 7 dias: [3.0463914  3.04579395 2.91154482 3.04459905 3.0440016  3.04340415
 3.0428067 ]
